# Linear Regression

## Using GLM

In [1]:
using GLM
using RDatasets
using MLDataUtils

### Load data

In [2]:
data = RDatasets.dataset("datasets", "mtcars")
first(data, 6)

,Model,MPG,Cyl,Disp,HP,DRat,WT,QSec,VS
,String31,Float64,Int64,Float64,Int64,Float64,Float64,Float64,Int64
1,Mazda RX4,21.0,6,160.0,110,3.9,2.62,16.46,0
2,Mazda RX4 Wag,21.0,6,160.0,110,3.9,2.875,17.02,0
3,Datsun 710,22.8,4,108.0,93,3.85,2.32,18.61,1
4,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1
5,Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02,0
6,Valiant,18.1,6,225.0,105,2.76,3.46,20.22,1


### Training/Testing set

In [3]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(data)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [4]:
train = data[train_ind, :]
test = data[test_ind, :]

,Model,MPG,Cyl,Disp,HP,DRat,WT,QSec,VS
,String31,Float64,Int64,Float64,Int64,Float64,Float64,Float64,Int64
1,Lotus Europa,30.4,4,95.1,113,3.77,1.513,16.9,1
2,Cadillac Fleetwood,10.4,8,472.0,205,2.93,5.25,17.98,0
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1
4,Fiat X1-9,27.3,4,79.0,66,4.08,1.935,18.9,1
5,Dodge Challenger,15.5,8,318.0,150,2.76,3.52,16.87,0
6,Merc 240D,24.4,4,146.7,62,3.69,3.19,20.0,1


### Model

In [5]:
ols = GLM.lm(@formula(MPG ~ Cyl + Disp + HP + DRat + WT + QSec + VS + AM + Gear + Carb), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

MPG ~ 1 + Cyl + Disp + HP + DRat + WT + QSec + VS + AM + Gear + Carb

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  -4.30523    22.8221     -0.19    0.8529  -52.9495     44.339
Cyl           0.512068    1.21665     0.42    0.6798   -2.08116     3.1053
Disp          0.0104783   0.0223627   0.47    0.6461   -0.0371867   0.0581434
HP           -0.0194899   0.0259786  -0.75    0.4647   -0.0748621   0.0358822
DRat          2.17801     2.18098     1.00    0.3338   -2.47064     6.82667
WT           -3.35272     2.11268    -1.59    0.1334   -7.85579     1.15035
QSec          1.41866     0.852285    1.66    0

### Prediction

In [6]:
predict(ols, test)

6-element Vector{Union{Missing, Float64}}:
 24.283352226100675
 12.891891875651798
 20.209156428742812
 27.816116395500526
 16.352650164670038
 20.47335976633168

### Validation

In [7]:
GLM.r²(ols)

0.8588256478581886

## Using MLJ

In [8]:
using MLJ

┌ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1423
┌ Warning: Module Distributions with build ID 106535861419595 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module Distributions with build ID 106535861419595 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module Distributions with build ID 106535861419595 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module Distributions with build ID 106535861419595 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969a

### Casting scientific types

In [9]:
y, X = unpack(data[!, 2:end], ==(:MPG), colname -> true);
first(X, 6)

,Cyl,Disp,HP,DRat,WT,QSec,VS,AM,Gear,Carb
,Int64,Float64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64
1,6,160.0,110,3.9,2.62,16.46,0,1,4,4
2,6,160.0,110,3.9,2.875,17.02,0,1,4,4
3,4,108.0,93,3.85,2.32,18.61,1,1,4,1
4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
5,8,360.0,175,3.15,3.44,17.02,0,0,3,2
6,6,225.0,105,2.76,3.46,20.22,1,0,3,1


In [10]:
first(X, 6) |> pretty

┌───────┬────────────┬───────┬────────────┬────────────┬────────────┬───────┬───────┬───────┬───────┐
│ Cyl   │ Disp       │ HP    │ DRat       │ WT         │ QSec       │ VS    │ AM    │ Gear  │ Carb  │
│ Int64 │ Float64    │ Int64 │ Float64    │ Float64    │ Float64    │ Int64 │ Int64 │ Int64 │ Int64 │
│ Count │ Continuous │ Count │ Continuous │ Continuous │ Continuous │ Count │ Count │ Count │ Count │
├───────┼────────────┼───────┼────────────┼────────────┼────────────┼───────┼───────┼───────┼───────┤
│ 6.0   │ 160.0      │ 110.0 │ 3.9        │ 2.62       │ 16.46      │ 0.0   │ 1.0   │ 4.0   │ 4.0   │
│ 6.0   │ 160.0      │ 110.0 │ 3.9        │ 2.875      │ 17.02      │ 0.0   │ 1.0   │ 4.0   │ 4.0   │
│ 4.0   │ 108.0      │ 93.0  │ 3.85       │ 2.32       │ 18.61      │ 1.0   │ 1.0   │ 4.0   │ 1.0   │
│ 6.0   │ 258.0      │ 110.0 │ 3.08       │ 3.215      │ 19.44      │ 1.0   │ 0.0   │ 3.0   │ 1.0   │
│ 8.0   │ 360.0      │ 175.0 │ 3.15       │ 3.44       │ 17.02      │ 0.0   │ 0.0 

In [11]:
X = coerce(X, :Cyl => Continuous, :HP => Continuous, :VS => Continuous, :AM => Continuous,
              :Gear => Continuous, :Carb  => Continuous)
first(X, 6)

,Cyl,Disp,HP,DRat,WT,QSec,VS,AM,Gear
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.0,160.0,110.0,3.9,2.62,16.46,0.0,1.0,4.0
2,6.0,160.0,110.0,3.9,2.875,17.02,0.0,1.0,4.0
3,4.0,108.0,93.0,3.85,2.32,18.61,1.0,1.0,4.0
4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0
5,8.0,360.0,175.0,3.15,3.44,17.02,0.0,0.0,3.0
6,6.0,225.0,105.0,2.76,3.46,20.22,1.0,0.0,3.0


### Training/testing set

In [12]:
train, test = partition(eachindex(y), 0.7, shuffle=true)

([10, 17, 26, 32, 28, 31, 20, 22, 30, 4  …  7, 15, 12, 11, 9, 23, 25, 24, 14, 21], [27, 3, 18, 6, 8, 16, 1, 2, 13, 5])

### Model

In [13]:
LinearRegressor = @load LinearRegressor pkg=GLM

import MLJGLMInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/yuehhua/.julia/packages/MLJModels/lDzCR/src/loading.jl:168
┌ Info: Precompiling MLJGLMInterface [caf8df21-4939-456d-ac9c-5fefbfb04c0c]
└ @ Base loading.jl:1423
┌ Warning: Module Distributions with build ID 106548130345052 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Info: Skipping precompilation since __precompile__(false). Importing MLJGLMInterface [caf8df21-4939-456d-ac9c-5fefbfb04c0c].
└ @ Base loading.jl:1124


MLJGLMInterface.LinearRegressor

In [14]:
linreg = machine(LinearRegressor(), X, y)

Machine trained 0 times; caches data
  model: LinearRegressor(fit_intercept = true, …)
  args: 
    1:	Source @024 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @210 ⏎ `AbstractVector{Continuous}`


### Training

In [15]:
fit!(linreg, rows=train)

┌ Info: Training machine(LinearRegressor(fit_intercept = true, …), …).
└ @ MLJBase /home/yuehhua/.julia/packages/MLJBase/rQDaq/src/machines.jl:487


Machine trained 1 time; caches data
  model: LinearRegressor(fit_intercept = true, …)
  args: 
    1:	Source @024 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @210 ⏎ `AbstractVector{Continuous}`


### Predict

In [16]:
ŷ = predict_mean(linreg, rows=test)

10-element Vector{Float64}:
 24.466960164672948
 26.802346409866516
 28.170476139355667
 21.161348109490117
 21.67362012227425
 11.51068469138227
 23.323655951374413
 22.81582371972778
 15.569168189314965
 17.436866654048437

### Evaluation

In [17]:
rms(ŷ, y[test])

2.59969916232371